In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
# import seaborn as sns

from m import *


%matplotlib inline
sns.set_style("whitegrid")

#df= pd.read_csv('/data/DS_INTERN/data/RAW_DATA/train_test_data.csv', low_memory=False)
df = pd.read_csv('/data/DS_INTERN/data/RAW_DATA/train_test_data_new.csv', low_memory=False)

df1 = add_target_column(df)

train_df , test_df = train_test_split(df1)

train_df = perform_preprocessing(train_df)
test_df = perform_preprocessing(test_df)

In [2]:
train_df = data_augmentation_simple_walmart(train_df,n_per_original=2)

100%|██████████| 50248/50248 [02:35<00:00, 322.50it/s]


In [3]:
len(train_df)

69935

In [4]:
from fuzzy_fns import *

In [5]:
# import time
# start = time.time()
# partial = False
# ratio_type = 'sort'

# df_result = (train_df.pipe(fuzzy_tf_idf_1, # Function and messy data
#                      column = 'FEAT1', # Messy column in data
#                      clean = (test_df['FEAT1']), # Master data (list)
#                      mapping_df = train_df, # Master data
#                      col = 'Result', # Can be customized
#                      partial = partial, 
#                      ratio_type = ratio_type
#                           )
#             )

# print(time.time() - start)

In [6]:
def fuzzy_nn_match1(
    messy,
    clean,
    column,
    col,partial, ratio_type,
    n_neighbors = 100,
    limit = 1,  **kwargs):
    nearest_values, _ = tfidf_nn(messy, clean, n_neighbors, **kwargs)

    results = [find_matches_fuzzy(row, nearest_values[i], partial, ratio_type, limit ) for i, row in enumerate(messy)]
    
    return ((results))

In [8]:
%%time

import time
start=time.time()

# dff = fuzzy_tf_idf_23(train_df[:3], 'FEAT1', test_df['FEAT1'], test_df, 'Result')

dff = fuzzy_nn_match1(test_df['FEAT1'], train_df['FEAT1'],'FEAT1', 'Result', partial=False, ratio_type='sort')

print(time.time()-start)

2360.272367477417
CPU times: user 37min 15s, sys: 1min 53s, total: 39min 9s
Wall time: 39min 20s


In [9]:
dff

[[('ORG PEMP ORG AVOCADO BAG NUT MEAT NUT WALNUT WALNUT TRAY',
   'BIOLOGICAL CRANBERRY WALNUT ORG CRANBERRY WALNUT ORG NUT NUT MEAT NUT SHELLED WALNUT',
   64)],
 [('DRIED GUAJILLO DRIED GUAJILLO REPL FRM DRIED GUAJIL MISC FLORAL FLORAL ORGANICOS',
   'DRIED GUAJILLO CHILE CHILI DRY GUAJILLO DRIED GUAJILLO ETHNIC SPECIALTY HISPANIC ETHNIC SPECIALTY PKG DRY CHILI',
   58)],
 [('CHERISH FRUIT BASKET CHERISH FRUIT BASKET FLORAL MISC FLORAL PREMADE FRUIT BASKET',
   'FRUIT BURST FRUIT BURST DSD CHR DIV FRUIT BURST CUT FRUIT PROCESSED CUT FRUIT SMALL MIX PERIODIC BLEND',
   58)],
 [('CHERISH FRUIT BASKET CHERISH FRUIT BASKET FLORAL MISC FLORAL PREMADE FRUIT BASKET',
   'FRUIT BURST FRUIT BURST DSD CHR DIV FRUIT BURST CUT FRUIT PROCESSED CUT FRUIT SMALL MIX PERIODIC BLEND',
   58)],
 [('DELUXE HOLIDAY FRUIT BASKET DELUXE HLDY BASKET CAITO DEX HDY BASKET MISC FLORAL PREMADE FRUIT BASKET',
   'HANGING BASKET HLDY BASKET HOLIDAY BASKET BROWN BASKET MIXED BOUQUET MIXED BOUQUET EVERGREEN',
   62

In [10]:
major_list = []

for row in range(len(dff)):
    
    major_list.append(pd.DataFrame(dff[row], columns = ['FEAT1_test','FEAT1_train', 'Ratio']))

In [11]:
major_df = pd.concat(major_list)
major_df.reset_index(inplace=True)
major_df.drop(['index'],axis=1, inplace=True)

In [57]:
len(major_df[major_df['Ratio']>=50])

223808

In [58]:
len(major_df)

244187

In [74]:
major_df.head()

,FEAT1_test,FEAT1_train,Ratio
0,ORG PEMP ORG AVOCADO BAG NUT MEAT NUT WALNUT W...,BIOLOGICAL CRANBERRY WALNUT ORG CRANBERRY WALN...,64
1,DRIED GUAJILLO DRIED GUAJILLO REPL FRM DRIED G...,DRIED GUAJILLO CHILE CHILI DRY GUAJILLO DRIED ...,58
2,CHERISH FRUIT BASKET CHERISH FRUIT BASKET FLOR...,FRUIT BURST FRUIT BURST DSD CHR DIV FRUIT BURS...,58
3,CHERISH FRUIT BASKET CHERISH FRUIT BASKET FLOR...,FRUIT BURST FRUIT BURST DSD CHR DIV FRUIT BURS...,58
4,DELUXE HOLIDAY FRUIT BASKET DELUXE HLDY BASKET...,HANGING BASKET HLDY BASKET HOLIDAY BASKET BROW...,62


In [76]:
xx = test_df.reset_index(drop=True).join(major_df)

In [78]:
xx.shape

(244187, 6)

In [79]:
xx.h

,ITEM1_DESC,FEAT1,Predicted_target,FEAT1_test,FEAT1_train,Ratio
0,ORG 60 10 3 PEMP,ORG PEMP ORG AVOCADO BAG NUT MEAT NUT WALNUT W...,NUT MEATS_SHELLED WALNUTS,ORG PEMP ORG AVOCADO BAG NUT MEAT NUT WALNUT W...,BIOLOGICAL CRANBERRY WALNUT ORG CRANBERRY WALN...,64
1,PR DRIED GUAJILLO,DRIED GUAJILLO DRIED GUAJILLO REPL FRM DRIED G...,ETHNIC AND SPECIALTY_DRY CHILIS,DRIED GUAJILLO DRIED GUAJILLO REPL FRM DRIED G...,DRIED GUAJILLO CHILE CHILI DRY GUAJILLO DRIED ...,58
2,CHERISH FRUIT BASKET PB,CHERISH FRUIT BASKET CHERISH FRUIT BASKET FLOR...,CUT FRUIT PROCESSED_SEASONAL BLEND,CHERISH FRUIT BASKET CHERISH FRUIT BASKET FLOR...,FRUIT BURST FRUIT BURST DSD CHR DIV FRUIT BURS...,58
3,CHERISH FRUIT BASKET PB,CHERISH FRUIT BASKET CHERISH FRUIT BASKET FLOR...,CUT FRUIT PROCESSED_SEASONAL BLEND,CHERISH FRUIT BASKET CHERISH FRUIT BASKET FLOR...,FRUIT BURST FRUIT BURST DSD CHR DIV FRUIT BURS...,58
4,DELUXE HLDY BASKET CAITO,DELUXE HOLIDAY FRUIT BASKET DELUXE HLDY BASKET...,MIXED BOUQUETS_MIXED BOUQUETS,DELUXE HOLIDAY FRUIT BASKET DELUXE HLDY BASKET...,HANGING BASKET HLDY BASKET HOLIDAY BASKET BROW...,62


In [60]:
len(test_df)

244187

In [61]:
FEAT1_TARGET_mapping_train = dict(zip(train_df.FEAT1, train_df.TARGET))

In [73]:
for row in range(len(major_df)):
    
    data_row = major_df.iloc[row]
    
    if data_row['Ratio'] >=50:
        test_df.iloc[row]['Predicted_target'] = FEAT1_TARGET_mapping_train[data_row['FEAT1_train']]

KeyboardInterrupt: 

In [19]:
# test_df['Predicted_target'].describe()

In [52]:
# test_df.rename(columns={'TARGET':'Predicted_target'}, inplace= True)

In [63]:
tgh = test_df.dropna()

In [64]:
tgi = trust_factor(tgh)

/libs/base/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/jupyter/m.py:643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_df['category'] = pd.Series(perform_spell_correction_manual(test_df['category']), index=test_df.index)
/home/jupyter/m.py:644: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [65]:
tgi.describe()

,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
count,223803.000000,223803.000000,223803.000000,223803.000000,223803.000000
mean,90.016823,91.163300,0.816933,0.829922,0.902861
std,21.809221,20.143737,0.386723,0.375702,0.296148
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,100.000000,100.000000,1.000000,1.000000,1.000000
50%,100.000000,100.000000,1.000000,1.000000,1.000000
75%,100.000000,100.000000,1.000000,1.000000,1.000000
max,100.000000,100.000000,1.000000,1.000000,1.000000


In [56]:
tgi

,ITEM1_DESC,FEAT1,Predicted_target,category,variety,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
6537,ORG 60 10 3 PEMP,ORG PEMP ORG AVOCADO BAG NUT MEAT NUT WALNUT W...,NUT MEATS_SHELLED WALNUTS,NUT MEAT,SHELLED WALNUT,100,100,1,1,1
8606,DELUXE HLDY BASKET CAITO,DELUXE HOLIDAY FRUIT BASKET DELUXE HLDY BASKET...,MIXED BOUQUETS_MIXED BOUQUETS,MIXED BOUQUET,MIXED BOUQUET,38,38,0,0,0
8607,D1 FRUIT BASKET LRG,LARGE FRUIT BASKET LRG FRUIT BASKET LRG DSD LA...,CUT FRUIT PROCESSED_SEASONAL BLEND,CUT FRUIT PROCESSED,SEASONAL BLEND,100,36,1,0,1
8609,FRUIT BOWL RM STN,READY MADE CHECKER FRUIT BOWL FRUIT BOWL STN A...,CUT FRUIT PROCESSED_FRUIT BOWL,CUT FRUIT PROCESSED,FRUIT BOWL,100,100,1,1,1
8610,D1 FRUIT BASKET SM,SMALL FRUIT BASKET FRUIT BASKET DSD LANCASTER ...,CUT FRUIT PROCESSED_SEASONAL BLEND,CUT FRUIT PROCESSED,SEASONAL BLEND,100,43,1,0,1
8611,BQ COP MIX BQT PG,COP MIXED BQT COP MIX BQT COP MIXED BQT FLORAL...,MIXED BOUQUETS_MIXED BOUQUETS,MIXED BOUQUET,MIXED BOUQUET,100,100,1,1,1
8612,CB COP LEATHER LEAF,CUSTOMER ORDERED PROGRAM COP LEATHER LEAF CUST...,CONSUMER BUNCHES_CONSUMER BUNCHES,CONSUMER BUNCH,CONSUMER BUNCH,36,36,0,0,0
8613,CB COP MLLNSTAR GYP,CUSTOMER ORDERED PROGRAM COP MLLNSTAR GYP CUST...,CONSUMER BUNCHES_CONSUMER BUNCHES,CONSUMER BUNCH,CONSUMER BUNCH,36,36,0,0,0
8614,RS COPHT PNK ROSE,CUSTOMER ORDERED PROGRAM COPHT PNK ROSE CUSTOM...,CONSUMER BUNCHES_CONSUMER BUNCHES,CONSUMER BUNCH,CONSUMER BUNCH,36,36,0,0,0
8615,RS COPLT PNK ROSE,CUSTOMER ORDERED PROGRAM COPLT PNK ROSE CUSTOM...,CONSUMER BUNCHES_CONSUMER BUNCHES,CONSUMER BUNCH,CONSUMER BUNCH,36,36,0,0,0
